In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('data/Earthquake.csv')
df.index.name = 'index'
df.head()

,日期,时间,维度,经度,方向,距离,深度,烈度
index,,,,,,,,
0,2003.05.20,12:17:44 AM,39.04,40.38,west,0.1,10.0,0.0
1,2007.08.01,12:03:08 AM,40.79,30.09,west,0.1,5.2,4.0
2,1978.05.07,12:41:37 AM,38.58,27.61,south_west,0.1,0.0,0.0
3,1997.03.22,12:31:45 AM,39.47,36.44,south_west,0.1,10.0,0.0
4,2000.04.02,12:57:38 AM,40.80,30.24,south_west,0.1,7.0,0.0


In [3]:
df_pivoted = pd.pivot_table(df,index=df.index,columns='方向',values=df.columns[-3:],
                            fill_value='-').stack(level=0)
df_final = df[df.columns[:4]].join(df_pivoted.reset_index(level=1)).rename(columns={'level_1':'地震参数'})
df_final.head()

,日期,时间,维度,经度,地震参数,east,north,north_east,north_west,south,south_east,south_west,west
index,,,,,,,,,,,,,
0,2003.05.20,12:17:44 AM,39.04,40.38,深度,-,-,-,-,-,-,-,10
0,2003.05.20,12:17:44 AM,39.04,40.38,烈度,-,-,-,-,-,-,-,0
0,2003.05.20,12:17:44 AM,39.04,40.38,距离,-,-,-,-,-,-,-,0.1
1,2007.08.01,12:03:08 AM,40.79,30.09,深度,-,-,-,-,-,-,-,5.2
1,2007.08.01,12:03:08 AM,40.79,30.09,烈度,-,-,-,-,-,-,-,4


In [4]:
df_melted = df_final.reset_index().melt(id_vars=df_final.reset_index().columns[:6]
                ,value_vars=df_final.columns[-8:],var_name='方向',value_name='数量')
df_melted = df_melted[df_melted['数量']!='-']
df_pivoted = df_melted.pivot(index='index',columns='地震参数',values='数量').astype('float64')
result = (df_melted.iloc[:,[0,1,2,3,4,6]][~df_melted.duplicated('index')].sort_values(
            by='index').join(df_pivoted,on='index')).set_index('index').reindex(range(df.shape[0]))
result = result.iloc[:,:5].join(result.iloc[:,[7,5]]).join(result.iloc[:,6])
result.head()

,日期,时间,维度,经度,方向,距离,深度,烈度
index,,,,,,,,
0,2003.05.20,12:17:44 AM,39.04,40.38,west,0.1,10.0,0.0
1,2007.08.01,12:03:08 AM,40.79,30.09,west,0.1,5.2,4.0
2,1978.05.07,12:41:37 AM,38.58,27.61,south_west,0.1,0.0,0.0
3,1997.03.22,12:31:45 AM,39.47,36.44,south_west,0.1,10.0,0.0
4,2000.04.02,12:57:38 AM,40.80,30.24,south_west,0.1,7.0,0.0


In [5]:
df.equals(result)

True